In [2]:
import torch     # PyTorch
import datasets  # Huggingface Datasets; actual data

from IPython.display import display  # image display
import matplotlib.pyplot as plt      # image display
%matplotlib inline                   
import random
#Torchvision functions
from torchvision.transforms import ToTensor
import torchvision.transforms.functional as functional
import torchvision.transforms as transforms

In [3]:
dataset = datasets.load_dataset("DrishtiSharma/Anime-Face-Dataset")

Found cached dataset parquet (/Users/boranollana/.cache/huggingface/datasets/DrishtiSharma___parquet/DrishtiSharma--Anime-Face-Dataset-1886e0ceeb6ab6bb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

dont run the next block, torch.(con)cat fucks up with large tensors. use tim's

In [ ]:
#loading all images from dictionary to list
#all_images = dataset['train'][:63565]['image']
#resizing the first image
#resized_image = functional.resize(all_images[0], size=[64])
#saving the image to a torch tensor
#train_tensor = train_tensor = ToTensor()(resized_image).unsqueeze(0)
#for the rest of the images, resize then concatinate to that tensor, using dimension 0.
for i in range(27528, len(all_images)):
    resized_image = functional.resize(all_images[i], size=[64])
    train_tensor = torch.cat((train_tensor,ToTensor()(resized_image).unsqueeze(0)), 0)
    #if i%10000 == 0:  #I used these as a sanity check for my loop, ignore
    #    print('i = ', i)

#Saving the tensor so we dont run the above code multiple times.
torch.save(train_tensor, 'images.pt')
#If everything went well, we will see a tensor of size [63565, 3, 64, 64]:
train_tensor

In [4]:
# Set the output size
output_size = (64, 64)

# Define a transform to resize the images
transform = transforms.Compose([
    transforms.Resize(output_size),
    transforms.ToTensor(),
])

# Create an empty list to store the resized images as tensors
resized_images = []

# Loop through each image in the dataset and resize it
for i in range( len(dataset['train']) ):
    image = dataset['train'][i]['image']  # Get the i-th image and its label
    image_resized = transform(image)  # Resize the image
    resized_images.append(image_resized)  # Add the resized image to the list
    
    if i % 1000 == 0:
        print("Making progress at ",i,"out of ",len(dataset['train']))

# Convert the list of tensors to a tensor stack
resized_images_tensor = torch.stack(resized_images)

Making progress at  0 out of  63565
Making progress at  1000 out of  63565
Making progress at  2000 out of  63565
Making progress at  3000 out of  63565
Making progress at  4000 out of  63565
Making progress at  5000 out of  63565
Making progress at  6000 out of  63565
Making progress at  7000 out of  63565
Making progress at  8000 out of  63565
Making progress at  9000 out of  63565
Making progress at  10000 out of  63565
Making progress at  11000 out of  63565
Making progress at  12000 out of  63565
Making progress at  13000 out of  63565
Making progress at  14000 out of  63565
Making progress at  15000 out of  63565
Making progress at  16000 out of  63565
Making progress at  17000 out of  63565
Making progress at  18000 out of  63565
Making progress at  19000 out of  63565
Making progress at  20000 out of  63565
Making progress at  21000 out of  63565
Making progress at  22000 out of  63565
Making progress at  23000 out of  63565
Making progress at  24000 out of  63565
Making progre

In [7]:
#saving tensor
torch.save(resized_images_tensor, 'images.pt')
resized_images_tensor.shape

torch.Size([63565, 3, 64, 64])

In [8]:
#loading tensor:
data = torch.load('./images.pt')
data.shape

torch.Size([63565, 3, 64, 64])

In [15]:
#additional imports:
from torch.utils.data import DataLoader

In [19]:
#some hyperparam init:
bs = 5000

For GANs we dont need a train/test split.

In [22]:
#creating dataloader from our tensor
images = DataLoader(data, batch_size=bs, shuffle=False)
#batch shape:
next(iter(images)).shape

torch.Size([5000, 3, 64, 64])

Our discriminator is basically a CNN, whereas our generator is basically the equivalent of a cnn but instead of convolutions, we have the gradients of those convolutions with respect to our inputs;

Discriminator: Conv2d
Generator: ConvTranspose2d


In [30]:
#new dependency:
from torch import nn as nn

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        nn.Sequential(
            #3 channels to 64, 
            #kernel 3x3 so filters: 3x3x3, 
            #stride(slide) 1, 
            #no padding needed since our images are already preprocessed and centered.

            #gonna add more convolutions so it has a bigger viewing window, check ipad drawing for reference
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(64), #batch normalization size -> same as image size
            nn.ReLU(),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.Conv2d(256, 1),
            nn.Flatten(),#required output: some_sizex1x1
            nn.Sigmoid()#either pass the discriminator or not.0/1 Tanh can work as well
        )
    #forward pass, calls the initalizer
    def forward(self, input):
        output = self.nn.Sequential(input)
        return output

In [29]:
some_size = 256 #same output from discriminator: 256x1x1
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        nn.Sequential(
            nn.ConvTranspose2d(some_size, 128, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(64), #batch normalization size -> same as image size
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=0),
            nn.Sigmoid() #either this or tanh can work as well, maybe tune this?
        )
        
    def forward(self, input):
        output = self.nn.Sequential(input)
        return output

Notes to self: Re-check prof. alvarez notes for convtranspose2d.
Great additional article for convtranspose2d: https://medium.com/apache-mxnet/transposed-convolutions-explained-with-ms-excel-52d13030c7e8